<a href="https://colab.research.google.com/github/richapatel93/mobile-price-regression/blob/main/6_16DEEP_LEARNING_TO_PERFORM_SCALAR_REGRESSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#COSC 522 UTK
#Project 2
#Student Name: __Richa Patel_______

In [1]:
#load in data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression

#Show fewer digits to improve readability
np.set_printoptions(precision=3, suppress=True)


In [2]:
dataset = pd.read_csv("/ndtv_data_final.csv")
print(dataset.columns)

Index(['Unnamed: 0', 'Name', 'Brand', 'Model', 'Battery capacity (mAh)',
       'Screen size (inches)', 'Touchscreen', 'Resolution x', 'Resolution y',
       'Processor', 'RAM (MB)', 'Internal storage (GB)', 'Rear camera',
       'Front camera', 'Operating system', 'Wi-Fi', 'Bluetooth', 'GPS',
       'Number of SIMs', '3G', '4G/ LTE', 'Price'],
      dtype='object')


In [3]:
dataset.tail()

,Unnamed: 0,Name,Brand,Model,Battery capacity (mAh),Screen size (inches),Touchscreen,Resolution x,Resolution y,Processor,...,Rear camera,Front camera,Operating system,Wi-Fi,Bluetooth,GPS,Number of SIMs,3G,4G/ LTE,Price
1354,1354,Intex Aqua A2,Intex,Aqua A2,1500,4.0,Yes,480,800,4,...,5.0,0.3,Android,Yes,Yes,Yes,2,Yes,No,2599
1355,1355,Videocon Infinium Z51 Nova+,Videocon,Infinium Z51 Nova+,2000,5.0,Yes,480,854,4,...,8.0,5.0,Android,Yes,Yes,Yes,2,Yes,No,2940
1356,1356,Intex Aqua Y4,Intex,Aqua Y4,1700,4.5,Yes,480,854,2,...,5.0,2.0,Android,Yes,Yes,No,2,Yes,No,2999
1357,1357,iBall Andi4 B20,iBall,Andi4 B20,1250,4.0,Yes,480,800,1,...,2.0,0.3,Android,Yes,Yes,Yes,2,Yes,No,2498
1358,1358,iBall Andi Avonte 5,iBall,Andi Avonte 5,2150,5.0,Yes,480,854,4,...,8.0,0.0,Android,Yes,Yes,Yes,2,Yes,No,3999


In [ ]:
### Your code goes here and below

Step 2: Choose 7–10 Features That Affect Phone Price

In [4]:
selected_features = [
    'Battery capacity (mAh)',
    'Screen size (inches)',
    'Touchscreen',
    'Resolution x',
    'Resolution y',
    'RAM (MB)',
    'Internal storage (GB)',
    'Rear camera',
    'Front camera',
    '4G/ LTE'
]


Step 3: Encode Categorical Features

In [7]:
# Apply one-hot encoding to 'Touchscreen' and '4G/ LTE'
dataset = pd.get_dummies(dataset, columns=['Touchscreen', '4G/ LTE'], drop_first=True)

# Show the updated column names
print(dataset.columns)

Index(['Unnamed: 0', 'Name', 'Brand', 'Model', 'Battery capacity (mAh)',
       'Screen size (inches)', 'Resolution x', 'Resolution y', 'Processor',
       'RAM (MB)', 'Internal storage (GB)', 'Rear camera', 'Front camera',
       'Operating system', 'Wi-Fi', 'Bluetooth', 'GPS', 'Number of SIMs', '3G',
       'Price', 'Touchscreen_Yes', '4G/ LTE_Yes'],
      dtype='object')


Step 4: Normalize the Data

In [8]:
from sklearn.preprocessing import MinMaxScaler

# Step 4: Normalize the selected features
scaler = MinMaxScaler()

# Only normalize numeric columns — not the target (Price)
features_to_normalize = [
    'Battery capacity (mAh)', 'Screen size (inches)', 'Resolution x',
    'Resolution y', 'RAM (MB)', 'Internal storage (GB)',
    'Rear camera', 'Front camera'
]

# Apply normalization
dataset[features_to_normalize] = scaler.fit_transform(dataset[features_to_normalize])

# Check result
print(dataset[features_to_normalize].describe())


       Battery capacity (mAh)  Screen size (inches)  Resolution x  \
count             1359.000000           1359.000000   1359.000000   
mean                 0.386471              0.590063      0.297679   
std                  0.175053              0.137012      0.140993   
min                  0.000000              0.000000      0.000000   
25%                  0.258517              0.530612      0.250000   
50%                  0.398798              0.571429      0.250000   
75%                  0.498998              0.673469      0.437500   
max                  1.000000              1.000000      1.000000   

       Resolution y     RAM (MB)  Internal storage (GB)  Rear camera  \
count   1359.000000  1359.000000            1359.000000  1359.000000   
mean       0.332607     0.203148               0.059755     0.111761   
std        0.158460     0.139447               0.072177     0.082855   
min        0.000000     0.000000               0.000000     0.000000   
25%        0.27272

Step 5: Split the data into training, validation, and test sets.

In [10]:
# Correct feature list after encoding
selected_features = [
    'Battery capacity (mAh)',
    'Screen size (inches)',
    'Resolution x',
    'Resolution y',
    'RAM (MB)',
    'Internal storage (GB)',
    'Rear camera',
    'Front camera',
    'Touchscreen_Yes',
    '4G/ LTE_Yes'
]

# Now define features (X) and target (y)
X = dataset[selected_features]
y = dataset['Price']


In [11]:
from sklearn.model_selection import train_test_split

# Split into training (70%) and temporary (30%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the temp set into validation (15%) and test (15%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Print shapes to confirm
print("Training set:", X_train.shape)
print("Validation set:", X_val.shape)
print("Test set:", X_test.shape)


Training set: (951, 10)
Validation set: (204, 10)
Test set: (204, 10)


## Step 6 – Build an Overtrained Model

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Step 6: Build an overtrained model
over_model = Sequential()

# Input layer + hidden layer 1
over_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))

# Hidden layer 2
over_model.add(Dense(128, activation='relu'))

# Output layer (1 neuron for scalar output)
over_model.add(Dense(1))

# Compile the model
over_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model (intentionally long: 100 epochs)
history_over = over_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    verbose=1
)


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 403095904.0000 - mae: 12207.1221 - val_loss: 413602144.0000 - val_mae: 11776.8379
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 286786752.0000 - mae: 11154.7070 - val_loss: 413014464.0000 - val_mae: 11754.8271
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 275305504.0000 - mae: 11197.9844 - val_loss: 411185184.0000 - val_mae: 11686.7529
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 320573376.0000 - mae: 11579.3164 - val_loss: 407030688.0000 - val_mae: 11531.6074
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 322960320.0000 - mae: 11015.7031 - val_loss: 399461280.0000 - val_mae: 11244.2656
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 333898304.0000 - mae: 10835.0332 - val_loss: 387630048.0000 - val_mae: 10781.6387
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 290815488.0000 - mae: 10598.7812 - val_loss: 371380576.0000 - val_mae: 10117.3330
Epoch 8/100
30/30 ━━━━━━

 Step 7: Add Early Stopping

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Step 7: Setup early stopping
early_stop = EarlyStopping(
    monitor='val_loss',     # Watch the validation loss
    patience=5,             # Stop if no improvement for 5 epochs
    restore_best_weights=True  # Keep the best model, not the last one
)

# Define the smarter model (can use smaller layers)
smart_model = Sequential()
smart_model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
smart_model.add(Dense(64, activation='relu'))
smart_model.add(Dense(1))  # Scalar output for price

# Compile the model
smart_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model with early stopping
history_smart = smart_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 329445984.0000 - mae: 11375.7861 - val_loss: 413684320.0000 - val_mae: 11779.9434
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 307453120.0000 - mae: 11445.9873 - val_loss: 413525056.0000 - val_mae: 11774.0664
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 305025760.0000 - mae: 11659.8721 - val_loss: 413116320.0000 - val_mae: 11759.0449
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 296747840.0000 - mae: 11438.6777 - val_loss: 412261664.0000 - val_mae: 11727.6152
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 342643392.0000 - mae: 11599.9297 - val_loss: 410803008.0000 - val_mae: 11673.7402
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 294380192.0000 - mae: 11506.8926 - val_loss: 408502112.0000 - val_mae: 11588.3311
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 300764960.0000 - mae: 11055.8291 - val_loss: 405115136.0000 - val_mae: 11461.6260
Epoch 8/100
30/30 ━━━━━━

Step 8 – Evaluate the final model on the test set

In [14]:
# Step 8: Evaluate the final model on the test set
test_loss, test_mae = smart_model.evaluate(X_test, y_test, verbose=1)

print("✅ Final Test MAE (Mean Absolute Error): ₹", round(test_mae, 2))


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 93922600.0000 - mae: 5873.0068 
✅ Final Test MAE (Mean Absolute Error): ₹ 6078.78
